In [93]:
# #Import required libraries

# import matplotlib.pyplot as plt
# import numpy as np
# import os

# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
# from tensorflow.keras.models import Sequential 
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
import matplotlib.pyplot as plt
import random
import os
import pandas as pd

In [94]:
#Import train, validation, test

train_dir= os.listdir("./datasets/datasets/train")
val_dir = os.listdir("./datasets/datasets/val")
test_dir = os.listdir("./datasets/datasets/test")

## DATA PREPROCESSING

In [95]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        './datasets/datasets/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        './datasets/datasets/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [168]:
test_generator = train_datagen.flow_from_directory(
        './datasets/datasets/test',
    classes=['test'],
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 0 images belonging to 1 classes.


## BUILDING MODEL

In [97]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2) , padding='same'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2) , padding='same'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [98]:
model.get_weights()

[array([[[[-6.14376441e-02, -7.84701332e-02, -7.86340684e-02,
           -2.51859426e-04,  1.20201185e-01, -1.31539136e-01,
            1.19030431e-01,  8.84857178e-02,  1.19446561e-01,
            4.29454446e-03, -1.73466057e-02,  8.69764090e-02,
           -8.85151029e-02,  4.94242013e-02,  3.55631113e-04,
           -1.12270691e-01,  5.04156798e-02, -6.07207716e-02,
            5.02924323e-02,  3.99294645e-02,  3.69279385e-02,
           -5.69778457e-02,  6.02790266e-02, -9.13749337e-02,
            5.27303517e-02, -1.69091970e-02, -1.17278352e-01,
            4.17278409e-02,  1.78018212e-03,  1.88998878e-03,
           -1.08745232e-01, -2.30395123e-02],
          [ 1.71764791e-02,  7.40372837e-02, -8.35407078e-02,
           -1.00328915e-01,  6.32692873e-02,  6.92875683e-03,
            4.69525754e-02,  8.54588300e-02, -6.02969900e-02,
            1.19605318e-01, -1.37838721e-03, -7.29567856e-02,
            2.24844962e-02,  1.31976858e-01, -1.09133109e-01,
           -4.04288545e-

In [99]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
activation_30 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
activation_31 (Activation)   (None, 75, 75, 32)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 38, 38, 64)       

## TRAINING MODEL

In [103]:
#learning rate

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5, min_lr = 0.1)
callbacks = [earlystop,learning_rate_reduction]

In [104]:
model.fit(
        train_generator,
        steps_per_epoch= 3600// batch_size,
        epochs=11,
        validation_data=validation_generator,
        validation_steps= 1800// batch_size,
callbacks=callbacks)
# model.save_weights('final.h5')
# model.save_model('')
# model.save_weights('new_model.h5')  # always save your weights after training or during training

Epoch 1/11
112/112 [==============================] - 137s 1s/step - loss: 0.6745 - accuracy: 0.6091 - val_loss: 0.6107 - val_accuracy: 0.6730
Epoch 2/11


112/112 [==============================] - 120s 1s/step - loss: 0.6299 - accuracy: 0.6487 - val_loss: 0.6069 - val_accuracy: 0.6590
Epoch 3/11


112/112 [==============================] - 116s 1s/step - loss: 0.6097 - accuracy: 0.6752 - val_loss: 0.6096 - val_accuracy: 0.6484
Epoch 4/11


112/112 [==============================] - 115s 1s/step - loss: 0.5923 - accuracy: 0.6908 - val_loss: 0.5574 - val_accuracy: 0.7338
Epoch 5/11


112/112 [==============================] - 112s 998ms/step - loss: 0.5809 - accuracy: 0.6989 - val_loss: 0.5329 - val_accuracy: 0.7333
Epoch 6/11


...

112/112 [==============================] - 111s 991ms/step - loss: 0.5490 - accuracy: 0.7268 - val_loss: 0.5460 - val_accuracy: 0.7176
Epoch 8/11


112/112 [==============================] - 109s 972ms/step - loss: 0.5373 - accuracy: 0.7341 - val_loss: 0.4936 - val_accuracy: 0.7679
Epoch 9/11


112/112 [==============================] - 108s 962ms/step - loss: 0.5091 - accuracy: 0.7603 - val_loss: 0.7042 - val_accuracy: 0.7109
Epoch 10/11


112/112 [==============================] - 106s 943ms/step - loss: 0.5145 - accuracy: 0.7578 - val_loss: 0.5274 - val_accuracy: 0.7327
Epoch 11/11


112/112 [==============================] - 107s 952ms/step - loss: 0.5080 - accuracy: 0.7539 - val_loss: 0.4638 - val_accuracy: 0.7762


In [116]:
model.save_weights('final.h5') 

In [ ]:
model.save_model('')

In [130]:
test_df = pd.DataFrame({
    'filename': test_dir
})
nb_samples = test_df.shape[0]

In [132]:
predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))

In [134]:
predict

array([[0.6292957 ],
       [0.08865613],
       [0.38364977],
       [0.73003006],
       [0.7336165 ],
       [0.20771143],
       [0.36411083],
       [0.8725804 ],
       [0.72535527],
       [0.504427  ],
       [0.831676  ],
       [0.5272532 ],
       [0.6448842 ],
       [0.75793695],
       [0.6669121 ],
       [0.3489353 ],
       [0.99535525],
       [0.30222696],
       [0.5772266 ],
       [0.4573216 ],
       [0.6917672 ],
       [0.70322454],
       [0.67086756],
       [0.6385457 ],
       [0.89065754],
       [0.54103255],
       [0.9324193 ],
       [0.97654825],
       [0.6300627 ],
       [0.3388024 ],
       [0.75727403],
       [0.70767504],
       [0.10122377],
       [0.8399309 ],
       [0.52085215],
       [0.18993005],
       [0.07664961],
       [0.95168865],
       [0.95475984],
       [0.55729556],
       [0.4196074 ],
       [0.17285353],
       [0.25199625],
       [0.53844625],
       [0.6367109 ],
       [0.9939252 ],
       [0.05426967],
       [0.326

KeyError: 'binary'

In [156]:
from sklearn.metrics import classification_report
clr = classification_report(test.labels, predict, labels=[0, 1], target_names=["CAT", "DOG"])

NameError: name 'test' is not defined

In [172]:
import gzip
a_file = gzip. open("cifar-10-python.tar.gz", "rb")
contents = a_file.read()